# Answering Business Questions Using SQL

In [1]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /home/nbuser/anaconda3_420/lib/python3.5/site-packages (20.1.1)


In [2]:
!pip install ipython-sql

In [3]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Overview of the Data

Query the database to get a list of all tables and views in the database

In [4]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [5]:
%%sql
SELECT
    *
FROM customer
LIMIT 5;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [6]:
%%sql
SELECT
    *
FROM track
LIMIT 5;

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


## Selecting New Albums to purchase
Find the best selling genres in the U.S.

In [7]:
%%sql
WITH track_genre AS (
    SELECT
        c.customer_id,
        c.country,
        t.track_id,
        g.name AS genre
    FROM customer c
    JOIN invoice i ON c.customer_id = i.customer_id
    JOIN invoice_line il ON i.invoice_id = il.invoice_id
    JOIN track t ON il.track_id = t.track_id
    JOIN genre g ON t.genre_id = g.genre_id
    WHERE c.country = 'USA'
    ),
    genre_tracks_count AS (
        SELECT
            genre,
            COUNT(track_id) AS tracks_sold
        FROM track_genre
        GROUP BY genre
        ORDER BY tracks_sold DESC
    )
    
SELECT 
    genre,
    tracks_sold,
    ROUND(
        (CAST(tracks_sold AS FLOAT) / 
            (SELECT 
                CAST(SUM(tracks_sold) AS FLOAT)
                FROM genre_tracks_count) * 100)
        , 1) AS percent_sold
FROM genre_tracks_count

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percent_sold
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Latin,22,2.1
Pop,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


The record store has four albums to choose from but can only add three new records to the store. The choices are Regal (Hip-Hop), Red Tone (Punk), Meteor and the Girls (Pop) and Slim Jim Bites (Blues). To determine which records the store should choose we looked at what genres have sold the most records in the U.S. in the past. 12.4 of records sold were alternative and punk, 3.4 percent were the blues, 2.1 were pop, and 1.9 were hip-hop. Therefore, we have determined that the record store should select Red Tone, Meteor and the Girls, and Slim Jim Bites as the next three records to add to the store.

## Analyzing Employee Sales Performance

In [8]:
%%sql
SELECT
    e.employee_id,
    e.hire_date,
    e.first_name || " " || e.last_name AS employee_name,
    SUM(i.total) AS total_sales
FROM employee e
LEFT JOIN customer c ON e.employee_id = c.support_rep_id
LEFT JOIN invoice i ON c.customer_id = i.customer_id
WHERE e.title = 'Sales Support Agent'
GROUP BY e.employee_id, employee_name
ORDER BY total_sales DESC;

 * sqlite:///chinook.db
Done.


employee_id,hire_date,employee_name,total_sales
3,2017-04-01 00:00:00,Jane Peacock,1731.510000000004
4,2017-05-03 00:00:00,Margaret Park,1584.0000000000032
5,2017-10-17 00:00:00,Steve Johnson,1393.9200000000028


Jane Peackock is the highest seller and Steve Johnson is the lowest seller. However, total sales of the employees corresponds with thier length of employment. Once you take this into account, they have similar sales per length of employment.

## Analyzing Sales by Country

Create view that categorizes countries with less than 2 customers as 'Other'.

In [9]:
%%sql

DROP VIEW IF EXISTS vw_country;

CREATE VIEW vw_country AS
SELECT
    country,
    CASE 
        WHEN COUNT(c.country) > 1 THEN c.country
        ELSE 'Other'
    END AS country_modified,
    COUNT(c.country) AS count
FROM customer c
GROUP BY country
ORDER BY count DESC;

SELECT country, country_modified, count FROM vw_country;

 * sqlite:///chinook.db
Done.
Done.
Done.


country,country_modified,count
USA,USA,13
Canada,Canada,8
Brazil,Brazil,5
France,France,5
Germany,Germany,4
United Kingdom,United Kingdom,3
Czech Republic,Czech Republic,2
India,India,2
Portugal,Portugal,2
Argentina,Other,1


In [10]:
%%sql
WITH t AS 
    (
        SELECT 
        c.customer_id,
        v.country_modified AS country,
        i.invoice_id,
        i.total,
        CASE 
            WHEN v.country_modified = 'Other' THEN 1
            ELSE 0
        END AS sort
        FROM customer c
        LEFT JOIN vw_country v ON c.country = v.country
        LEFT JOIN invoice i ON c.customer_id = i.customer_id
    ),
    customer_sales AS 
    (
        SELECT 
            c.customer_id,
            c.country AS country,
            COUNT(i.invoice_id) AS number_of_sales,
            SUM(i.total) AS total_sales,
            SUM(i.total) / COUNT(i.invoice_id) average_value_of_sales
        FROM customer c
        LEFT JOIN invoice i ON c.customer_id = i.customer_id
        GROUP BY c.customer_id
    )

SELECT
    t.country,
    COUNT(distinct t.customer_id) AS total_customers,
    ROUND(SUM(t.total), 2) AS total_sales,
    ROUND(
            (SUM(t.total)/COUNT(distinct t.invoice_id))
        , 2) average_order_value,
    ROUND(
        (SUM(t.total)/COUNT(distinct t.customer_id))
        , 2) average_customer_value
FROM t
LEFT JOIN customer_sales cs ON t.customer_id = cs.customer_id
GROUP BY t.country
ORDER BY t.sort, total_sales DESC

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,average_order_value,average_customer_value
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.65
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.56
India,2,183.15,8.72,91.57
Other,15,1094.94,7.45,73.0


Based on average order value and average customer value, there may be opportunity in the following countries:

- Czech Republic  
- United Kingdom  
- India 

Since the number of customers in these countries is small, the data we ahve is limited. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Albums vs Individual Tracks

In [11]:
%%sql
WITH album_tracks AS 
    (
        SELECT
            a.album_id,
            t.track_id
        FROM track t
        LEFT JOIN album a ON t.album_id = a.album_id
        ORDER BY a.album_id, t.track_id
    ),
    invoice_tracks AS 
    (
        SELECT
            il.invoice_line_id,
            il.invoice_id,
            il.track_id,
            t.album_id
        FROM invoice_line il
        LEFT JOIN track t ON il.track_id = t.track_id
        ORDER BY il.invoice_id, il.invoice_line_id, t.album_id, il.track_id
    ),
    invoice_albums AS
    (
    SELECT
        i.invoice_id,
        i.album_id,
        i.track_id,
        CASE WHEN 
            (
                SELECT track_id FROM album_tracks WHERE album_tracks.album_id = a.album_id
                EXCEPT
                SELECT track_id FROM invoice_tracks WHERE invoice_tracks.invoice_id = i.invoice_id
            ) IS NULL
            AND
            (
                SELECT track_id FROM invoice_tracks WHERE invoice_tracks.invoice_id = i.invoice_id
                EXCEPT
                SELECT track_id FROM album_tracks WHERE album_tracks.album_id = a.album_id
            ) IS NULL
            THEN 1
            ELSE 0
            END AS album
    FROM invoice_tracks i 
    LEFT JOIN album_tracks a ON i.track_id = a.track_id
    ),
    album_purchase AS 
    (
        SELECT
            invoice_id,
            CASE 
                WHEN SUM(album) > 0 THEN 'Yes'
                ELSE 'No'
            END AS album_purchase
        FROM invoice_albums
        GROUP BY invoice_id
    )

SELECT 
    a.album_purchase, 
    COUNT(a.invoice_id) AS total_invoices,
    (CAST(COUNT(a.invoice_id) AS FLOAT)/(SELECT COUNT(*) FROM invoice) * 100) AS percent_invoices
    
FROM album_purchase a
GROUP BY a.album_purchase

 * sqlite:///chinook.db
Done.


album_purchase,total_invoices,percent_invoices
No,500,81.43322475570032
Yes,114,18.566775244299674


Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album. After analyzing the number of invoices with an album purchase, I would recommend that the Chinook store should continue to buy full albums. Nearly 20 percent of invoices had a full album and we would not want to lose that revenue.